In [2]:
import imblearn
import pandas as pd
import numpy as np
from sklearn import svm
import os
import gc
import math
from sklearn.utils import shuffle
os.environ["_CCTRACE_"]="ON";
import cloudComPy as cc;
cc.initCC();
import sys
import time
import matplotlib.pyplot as plt
import joblib
from imblearn.under_sampling import RandomUnderSampler
from cloudComPy import CloudSamplingTools
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

QSocketNotifier: Can only be used with threads started with QThread
/home/paul/projets/CloudComPy/CloudComPy/CloudCompare/libs/qCC_db/src/ccLog.cpp [48] : trace ON
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [966] : initializeConverters
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [152] : register QString_from_python_str
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [237] : register Vector2Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [237] : register Vector2Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [291] : register Vector2Tpl_from_python_list
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [291] : register Vector2Tpl_from_python_list
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [345] : register Vector3Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converter

In [3]:
# Data Preparation and cleaning
# Combining the labels and datasets
def combine_data(file):
    # Load data
    df = pd.read_csv('../Data/Test/'+file+'.txt', sep = ' ')
    label = pd.read_csv('../Data/Test/'+file+'.labels')
    # Drop the color columns
    df.columns = range(df.shape[1])
    df.drop(df.columns[[3, 4, 5, 6]], axis = 1, inplace = True)
    # Combine labels and datasets
    df[3] = label
    # Drop the rows with the Label 0
    #df.drop(df[4], axis=0, inplace =True)
    df.drop(df[(df[3] == 0)].index, inplace=True)
#     n = 200000  #chunk row size
#     list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
    df.to_csv('../Data/Test/'+file+'_combined.txt', index=False)
    return df

In [4]:
def split_dataframe(data):
    target = np.array(data['Label'])
    features = data.drop(['Label'], axis = 1)
    features = np.array(features)
    return target, features

In [5]:
# Splitting the datasets to allow balancing using the classes in the target field
def split_data(data):
    target = np.array(data[3])
    features = data.drop(data.columns[[3]], axis = 1)
    features = np.array(features)
    return target, features

In [61]:
# Computing geometric features
def compute_features(data):
    cloud = cc.loadPointCloud(data);
    cord = cloud.toNpArrayCopy();
    
    rad = 1.0
    #size = cloud.size()  
    #scale_half = round(size/2)
    # scale_quater = round(size/4)
    #scale_hunth = round(size/100)
    #sub_sample_half = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_half,cc.RESAMPLING_CELL_METHOD.CELL_CENTER)
    # sub_sample_quarter = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_quater,cc.RESAMPLING_CELL_METHOD.CELL_CENTER )
    #sub_sample_tenth = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_hunth,cc.RESAMPLING_CELL_METHOD.CELL_CENTER )
    clouds = [cloud]

    print("Computing ")
    cc.computeFeature(cc.GeomFeature.Anisotropy,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenEntropy,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue1,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue2,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue3,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValuesSum,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Linearity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Omnivariance ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.PCA1 ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.PCA2 ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Planarity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Sphericity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.SurfaceVariation ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Verticality ,rad,clouds)
    #cc.computeFeature(cc.GeomFeature.Verticality ,cc.GetPointCloudRadius(clouds),clouds)
    return sub_sample_tenth 

In [7]:
# Create a dataframe from a cloud input and return it
def cloud_to_dataframe(cloud_name):
    cloud = cloud_name;
    cord = cloud.toNpArrayCopy();
    
    df = pd.DataFrame(cord, columns = ['x','y','z'])
    print(df.shape)
    #fields_lists = ['Intensity','Label','Eigenvalues_sum','Omnivariance','Eigenentropy','Anisotropy','Planarity','Linearity','PCA1','PCA2','Surface_variation','Sphericity','Verticality','1st_eigenvalue','2nd_eigenvalue','3rd_eigenvalue']
    fields_lists = ['Label','Eigenvalues_sum','Omnivariance','Eigenentropy','Anisotropy','Planarity','Linearity','PCA1','PCA2','Surface_variation','Sphericity','Verticality','1st_eigenvalue','2nd_eigenvalue','3rd_eigenvalue']
    
    nsf = cloud.getNumberOfScalarFields()
    # Asign Columns names and values
    for i in range(nsf):
        df[fields_lists[i]] = cloud.getScalarField(i).toNpArrayCopy().tolist()
    # Drop rows with null values
    df.dropna(axis = 0,inplace = True)
    print(df['Label'].value_counts())
    return df

In [8]:
# Balancing the dataset using the target field
def bal_classes(feat, target,sel):
    if sel == 1:
        sm = SMOTE(random_state=42)
        X_r , y_r = sm.fit_resample(feat,target)
    else :
        us = RandomUnderSampler(sampling_strategy='not minority')
        X_r, y_r = us.fit_resample(feat, target)
#     X_r = shuffle(X_r, random_state=42)
#     y_r = shuffle(y_r, random_state=42)
    return X_r, y_r

In [9]:
# Recombining the target and features as a balanced dataset
def create_cloud_file(features, target, name):
    df_temp = pd.DataFrame(data=X_r)
    df_temp[3] = target
    df = df_temp.loc[:,0:4]
    
    df[[4,5,6]] = 0    
    for index, row in df.iterrows():
        if row[3] == 1:
            df.at[index,4] = 255
        elif row[3] == 2:
            df.at[index,5] = 255
        elif row[3] == 3:
            df.at[index,6] = 255
        elif row[3] == 4:
            df.at[index,4] = 255
            df.at[index,6] = 255
        elif row[3] == 5:
            df.at[index,4] = 255
            df.at[index,5] = 255
        elif row[3] == 6:
            df.at[index,5] = 255
            df.at[index,6] = 255
        elif row[3] == 7:
            df.at[index,4] = 192
            df.at[index,5] = 192
            df.at[index,6] = 192
        elif row[3] == 8:
            df.at[index,4] = 128
            df.at[index,6] = 128
    
    df.to_csv('../Data/Test/'+name+'_colored.txt', index=False)

    return df

In [9]:
# Steps for testing
# Combine files with labels
# Compute geometric features on the test data set
# Create a Data Frame from the cloud binary
# Load Models saved from training

In [10]:
# Combine the test file with its label
df = combine_data('untermaederbrunnen_station3_xyz_intensity_rgb')

In [ ]:
#df.shape
untermaederbrunnen_station3 = pd.read_csv('../Data/Test/bildstein_station5_xyz_intensity_rgb_combined.txt')
untermaederbrunnen_station3_sample = t.sample(frac=0.05, replace=True, random_state=1)
untermaederbrunnen_station3_sample.to_csv('untermaederbrunnen_station3_sample.txt', index=False)

In [70]:
t_new.shape

(816787, 4)

In [72]:
# Compute Geometric Features
cloud = compute_features('untermaederbrunnen_station3_sample.txt')


/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [432] : Opening file: untermaederbrunnen_station3_sample.txt mode: 0 x: 0 y: 0 z: 0 extraData: 
[CloudCompare] [I/O] File 'untermaederbrunnen_station3_sample.txt' loaded successfully
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [596] : Found one cloud with 816788 points
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ccPointCloudPy.cpp [185] : CoordsToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [756] : convertible to std::vector<ccHObject*>?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [759] : list ok
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [763] : nbobj 1
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [770] :   OK ccPointCloud0
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [794] : construct
/home/paul/projets/CloudComPy/CloudComPy/boostPython/conver

Computing 


cteristic 4 radius: 1
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [756] : convertible to std::vector<ccHObject*>?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [759] : list ok
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [763] : nbobj 1
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [770] :   OK ccPointCloud0
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [794] : construct
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [810] :   Extract OK ccPointCloud0
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [829] : pyCCComputeGeomCharacteristic 3 radius: 1


KeyboardInterrupt: 

In [16]:
cloud

In [59]:
# Create a DataFrame from the cloud of the test data
tdf = cloud_to_dataframe(cloud)

/home/paul/projets/CloudComPy/CloudComPy/boostPython/ccPointCloudPy.cpp [185] : CoordsToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudC

(106853, 3)
0.948148    160
0.984045    160
0.939394    119
0.939394    119
0.984045    110
           ... 
0.903404      1
0.910750      1
0.900548      1
0.905671      1
0.767495      1
Name: Label, Length: 87298, dtype: int64


In [60]:
tdf.head()

,x,y,z,Label,Eigenvalues_sum,Omnivariance,Eigenentropy,Anisotropy,Planarity,Linearity,PCA1,PCA2,Surface_variation,Sphericity,Verticality,1st_eigenvalue,2nd_eigenvalue
0,-84.670654,-104.858604,4.664719,0.903883,0.362633,0.093632,0.027369,0.009000,0.130001,0.707696,0.028464,0.720243,0.210530,0.196187,0.096117,0.069228,0.718538
1,-84.670654,-105.209541,5.015663,0.938083,0.435905,0.142531,0.034639,0.008825,0.185995,0.756975,0.035188,0.766317,0.186235,0.181108,0.061917,0.047448,0.905047
2,-85.021599,-104.858604,5.015663,0.903883,0.362633,0.093632,0.027369,0.009000,0.130001,0.707696,0.028464,0.720243,0.210530,0.196187,0.096117,0.069228,0.718538
3,-84.670654,-104.858604,5.015663,0.938083,0.435905,0.142531,0.034639,0.008825,0.185995,0.756975,0.035188,0.766317,0.186235,0.181108,0.061917,0.047448,0.905047
4,-84.319717,-105.209541,5.015663,0.938083,0.435905,0.142531,0.034639,0.008825,0.185995,0.756975,0.035188,0.766317,0.186235,0.181108,0.061917,0.047448,0.905047


In [42]:
tdf.head()

,x,y,z,Label,Eigenvalues_sum,Omnivariance,Eigenentropy,Anisotropy,Planarity,Linearity,PCA1,PCA2,Surface_variation,Sphericity,Verticality,1st_eigenvalue,2nd_eigenvalue
25,-23.731213,1.138645,-2.365940,0.851348,0.661530,0.203167,0.101373,0.030201,0.334741,0.501035,0.085362,0.606937,0.302840,0.350313,0.148652,0.090222,0.010321
26,-23.193157,-0.475525,-2.365940,0.797589,0.593166,0.147410,0.082715,0.029837,0.259962,0.438875,0.071388,0.567043,0.318182,0.358714,0.202411,0.114776,0.309976
27,-23.193157,0.062531,-2.365940,0.828428,0.626263,0.185331,0.081423,0.031798,0.298552,0.560663,0.078288,0.620768,0.272726,0.267765,0.171572,0.106506,0.187480
29,-23.731213,-11.774723,6.242973,0.635759,0.838725,0.231502,0.157912,0.084322,0.473736,0.317882,0.145536,0.488673,0.333333,0.317878,0.364241,0.177994,0.438724
30,-24.269272,-11.236664,5.704918,0.528888,0.624715,0.129146,0.072376,0.060842,0.262364,0.439576,0.082850,0.492238,0.275862,0.089312,0.471112,0.231899,0.305859


In [24]:
# Load models saved from training
rf = joblib.load('../Project/rf_model.joblib')
svc = joblib.load('../Project/svc.joblib')
knn = joblib.load('../Project/knn.joblib')


In [26]:
knn_best = joblib.load('../Project/knn_best.joblib')
svc_best = joblib.load('../Project/svc_best.joblib')
rf_best = joblib.load('../Project/rf_best.joblib')


In [44]:
# Split the dataframe 
target,features = split_dataframe(tdf)

# Apply Scaler Normalisation
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [45]:
features.shape

(8889, 16)

In [46]:
# Testing Accuracy, F1 and Jaccard Score
yhat_rf = rf.predict(features)
yhat_svm = svc.predict(features)
yhat_knn=knn.predict(features)


ValueError: X has 16 features, but RandomForestRegressor is expecting 18 features as input.

In [35]:
print(metrics.accuracy_score(target, yhat_knn))
print(metrics.accuracy_score(target, yhat_svm))
print(metrics.accuracy_score(target, yhat_rf))

0.2710727969348659
0.3668582375478927
0.3074712643678161


In [21]:
y_r.shape

(2088,)

In [41]:
#Random Forest
print("Jaccard: ", jaccard_score(y_r, yhat_rf,average='weighted'))
print("F1-score ", f1_score(y_r, yhat_rf, average='weighted'))

# SVC
print("Jaccard: ", jaccard_score(y_r, yhat_svm,average='weighted'))
print("F1-score ", f1_score(y_r, yhat_svm, average='weighted') )

# KNN
print("Jaccard: ", jaccard_score(y_r, yhat_knn,average='weighted'))
print("F1-score ", f1_score(y_r, yhat_knn, average='weighted') )


Jaccard:  0.1622223105478125
F1-score  0.26577183075883964
Jaccard:  0.19408951946239578
F1-score  0.30108023445523135
Jaccard:  0.14476684791763386
F1-score  0.24078527039806438


In [ ]:
# Color and cloud binary
#1: man-made terrain, 2: natural terrain, 3: high vegetation, 4: low vegetation, 5: buildings, 6: hard scape, 7: scanning artefacts, 8: cars}
#write out/save classified points, coloured by class label
#x,y,z, r, g, b
1 red=(255,0,0),
2 green=(0,255,0),
3 blue=(0,0,255,
4 magenta=(255,0,255),
5 yellow=(255,255,0),
6 cyan=(0,255,255),
7 silver=(192,192,192),
8 purple=(128,0,128)

In [45]:
create_dataframe(X_r,y_r,'test')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.337000,-28.389000,-0.593,-1327.0,3.0,0.118908,0.011715,0.009621,0.004011,0.025347,0.178706,0.007675,0.462172,0.379579,0.478890,0.342405,0.158250,0.971477
1,-1.725000,-37.408001,-1.942,-1360.0,6.0,0.051541,0.006308,0.002822,0.000384,0.009514,0.552617,0.001897,0.663042,0.296634,0.386566,0.060818,0.040325,0.948698
2,-12.335000,-18.853001,-0.886,-1167.0,2.0,0.110860,0.015814,0.009209,0.000256,0.025278,0.417670,0.003339,0.625589,0.364300,0.566168,0.016162,0.010111,0.080238
3,-1.722000,-28.388000,-0.010,-1176.0,3.0,0.081158,0.009958,0.003680,0.002431,0.016069,0.630462,0.004466,0.619690,0.228999,0.125366,0.244172,0.151311,0.205280
4,-2.032000,-37.870998,-2.318,-322.0,8.0,0.101421,0.011750,0.008410,0.001368,0.021528,0.284318,0.005132,0.545828,0.390639,0.599283,0.116398,0.063533,0.697673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,-19.542000,-36.491001,-2.209,-1521.0,7.0,0.109695,0.015580,0.009455,0.000083,0.025118,0.393107,0.002308,0.620252,0.376427,0.601539,0.005355,0.003321,0.010093
2084,-20.577999,-13.086000,-0.270,-875.0,5.0,0.075191,0.008644,0.005505,0.000764,0.014914,0.363139,0.003313,0.579623,0.369139,0.548462,0.088398,0.051238,0.993467
2085,-12.684000,-5.651000,0.296,-289.0,5.0,0.068192,0.009436,0.004467,0.000001,0.013904,0.526631,0.000391,0.678647,0.321251,0.473219,0.000150,0.000102,0.991382
2086,-20.146999,-18.389999,-0.020,-486.0,5.0,0.077720,0.013268,0.003616,0.000003,0.016888,0.727439,0.000517,0.785682,0.214147,0.272344,0.000218,0.000171,0.996147


In [111]:
df_temp.head()

,0,1,2,3,4,5,6,7
0,-1.337,-28.389000,-0.593,3.0,255,0,0,0
1,-1.725,-37.408001,-1.942,6.0,0,255,0,0
2,-12.335,-18.853001,-0.886,2.0,255,255,0,0
3,-1.722,-28.388000,-0.010,3.0,255,0,0,0
4,-2.032,-37.870998,-2.318,8.0,128,0,0,0


In [117]:
ds = create_cloud_file(X_r,y_r,'test2')

In [118]:
ds.head()

,0,1,2,3,4,5,6
0,-1.337,-28.389000,-0.593,3.0,0,0,255
1,-1.725,-37.408001,-1.942,6.0,0,255,255
2,-12.335,-18.853001,-0.886,2.0,0,255,0
3,-1.722,-28.388000,-0.010,3.0,0,0,255
4,-2.032,-37.870998,-2.318,8.0,128,0,128
